# Video Game Sales Data Analysis 
---
This Jupyter notebook will explore a dataset of international video game sales in order to identify patterns that determine whether a game succeeds or not.


#### Project Sections 
1. Initial Set Up and Data Preparation 
2. General Data Analysis 
3. Analysis By Sales Region 
4. Hypothesis Testing 

### Initial Set Up & Data Preparation 
---

In [12]:
# Import required libraries 
from scipy import stats as st
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt

# Read in Data
df = pd.read_csv('games.csv')

#### Data Cleaning 
Part 1: Finding Issues 

In [13]:
# Sample Data 
df.sample(3)

,Name,Platform,Year_of_Release,Genre,NA_sales,EU_sales,JP_sales,Other_sales,Critic_Score,User_Score,Rating
10737,NeverDead,X360,2012.0,Shooter,0.06,0.03,0.00,0.01,52.0,5.5,M
6349,Mass Effect 3,WiiU,2012.0,Role-Playing,0.14,0.11,0.00,0.02,NaN,NaN,NaN
346,SOCOM: U.S. Navy SEALs,PS2,2002.0,Shooter,2.53,0.81,0.06,0.24,82.0,7.9,M


In [14]:
# Check data types and Missing values 
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16715 entries, 0 to 16714
Data columns (total 11 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   Name             16713 non-null  object 
 1   Platform         16715 non-null  object 
 2   Year_of_Release  16446 non-null  float64
 3   Genre            16713 non-null  object 
 4   NA_sales         16715 non-null  float64
 5   EU_sales         16715 non-null  float64
 6   JP_sales         16715 non-null  float64
 7   Other_sales      16715 non-null  float64
 8   Critic_Score     8137 non-null   float64
 9   User_Score       10014 non-null  object 
 10  Rating           9949 non-null   object 
dtypes: float64(6), object(5)
memory usage: 1.4+ MB


Colunm names should be standardized to lowercase. Year_of_Release and User_Score are not in the correct datatype. User_score, Critic_Score, and Rating each have several null values, possibly because the games were not rated or critiqued. User and critic score should be numeric data types, so null values will be filled with filtered medians, while null ratings can be filled with a string like 'unrated'. 

In [15]:
# Check for duplicate rows 
df.duplicated().sum()

0

In [16]:
# Check for implicit duplicates
df['Platform'].unique()

array(['Wii', 'NES', 'GB', 'DS', 'X360', 'PS3', 'PS2', 'SNES', 'GBA',
       'PS4', '3DS', 'N64', 'PS', 'XB', 'PC', '2600', 'PSP', 'XOne',
       'WiiU', 'GC', 'GEN', 'DC', 'PSV', 'SAT', 'SCD', 'WS', 'NG', 'TG16',
       '3DO', 'GG', 'PCFX'], dtype=object)

In [17]:
df['Genre'].unique()

array(['Sports', 'Platform', 'Racing', 'Role-Playing', 'Puzzle', 'Misc',
       'Shooter', 'Simulation', 'Action', 'Fighting', 'Adventure',
       'Strategy', nan], dtype=object)

In [18]:
df['Rating'].unique()

array(['E', nan, 'M', 'T', 'E10+', 'K-A', 'AO', 'EC', 'RP'], dtype=object)

In [19]:
df['User_Score'].unique()

array(['8', nan, '8.3', '8.5', '6.6', '8.4', '8.6', '7.7', '6.3', '7.4',
       '8.2', '9', '7.9', '8.1', '8.7', '7.1', '3.4', '5.3', '4.8', '3.2',
       '8.9', '6.4', '7.8', '7.5', '2.6', '7.2', '9.2', '7', '7.3', '4.3',
       '7.6', '5.7', '5', '9.1', '6.5', 'tbd', '8.8', '6.9', '9.4', '6.8',
       '6.1', '6.7', '5.4', '4', '4.9', '4.5', '9.3', '6.2', '4.2', '6',
       '3.7', '4.1', '5.8', '5.6', '5.5', '4.4', '4.6', '5.9', '3.9',
       '3.1', '2.9', '5.2', '3.3', '4.7', '5.1', '3.5', '2.5', '1.9', '3',
       '2.7', '2.2', '2', '9.5', '2.1', '3.6', '2.8', '1.8', '3.8', '0',
       '1.6', '9.6', '2.4', '1.7', '1.1', '0.3', '1.5', '0.7', '1.2',
       '2.3', '0.5', '1.3', '0.2', '0.6', '1.4', '0.9', '1', '9.7'],
      dtype=object)

Part 2: Fixing Issues  
Upon initial exploration of the data, it has been determined that the following tasks must be carried out in order to prepare the data for analysis: 
- Column names should be made lowercase 
- Columns with object data types should have strings in lowercase
- year_of_release should be in datetime data type
- user_score should be a float data type
- Replace missing values: 
    - rating: Replace null with 'unrated'
    - critic_score and user_score: Replace null and 'tbd' with median of genre and platform
    - genre: replace null with 'misc'
    - name: drop null values (this is 0.01% of data)
    - year_of_release: drop rows with null values (this is 1.6% of data)
- Add a column with total sales 


In [20]:
# Make column names and object columns lowercase 
df = df.rename(
    columns={
        'Name':'name',
        'Platform':'platform',
        'Year_of_Release':'year_of_release',
        'Genre':'genre',
        ''
        
    }

)

### General Data Analysis
---
In this section, the entire dataset will be analyzed in order to answer some questions about global sales. The following questions will be answered: 
- How long does it generally take for new platforms to appear and old ones to fade?
- Which platforms are leading in sales? Which are growing and shrinking?
- Are the differences in global sales of games for each platform significant? What about average sales on various platforms? 
- Is there a correlation between user reviews and sales for the X platform?
- How do the sales of BLANK on BLANK compare to sales of the same game on BLANK?
- Which genres are the most profitable? Which are the least?

#### Is the data for every period significant?
Volume of games released per year will determine if every period in the data set is significant. 

#### How long does it generally take for new platforms to appear and old ones to fade?
To answer this question, sales variation from platform to platform will be analyzed to determine the platforms with the greatest total sales. Distributions will be built based on data from each year in order to analyze the rise and fall of these platforms. 

#### Which platforms are leading in sales? Which are growing and shrinking?

#### Are the differences in global sales of games for each significant? What about average sales on various platforms? 

#### Is there a correlation between user reviews and sales for the X platform?

#### How do the sales of BLANK on BLANK compare to sales of the same game on BLANK?

#### Which genres are the most profitable? Which are the least?

### Analysis by Sales Region
---
In this section, comparisons will be made between the three sales regions: North America, Europe, and Japan. The following factors will be analyzed:
- Platform market share variations by region
- Genre popularity variation across regions 
- Effect ESRB rating by region 

#### Platform Market Share Variations Across Regions

#### Genre Popularity Variations Across Regions

#### ESRB Rating Effect by Region 

### Hypothesis Testing 
---
This section will utilize hypothesis testing to see if the following two hypotheses are supported through statistical analysis:
1. Average user ratings of the Xbox One and PC platforms are the same. 
2. Average user ratings for the Action and Sports genres are different.

#### Are the average user ratings of the Xbox One and PC platforms the same?
- Null Hypothesis: There is a statistcally significant difference between the average user ratings of the Xbox One and PC platforms.
- Alternative Hypothesis: There is not a statistcally significant difference between the average user ratings of the Xbox One and PC platforms.

#### Are the average user ratings for the Action and Sports genres are different? 
- Null Hypothesis: There is not a statistcally significant difference between the average user ratings for the Action and Sports genres.
- Alternative Hypothesis: There is a statistcally significant difference between the average user ratings for the Action and Sports genres.



## Conclusions 
---